In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import time
import pickle
import datetime
import re
import pyActigraphy

import sys
sys.path.append('/home/ngrav/project/')
from wearables.scripts import utils as wearutils
from wearables.scripts import data_v42 as weardata

import torch

plt.rc('font', size = 9)
plt.rc('font', family='sans serif')
plt.rcParams['pdf.fonttype']=42
plt.rcParams['ps.fonttype']=42
plt.rcParams['legend.frameon']=False
plt.rcParams['axes.grid']=False
plt.rcParams['legend.markerscale']=0.5
plt.rcParams['savefig.dpi']=600
sns.set_style("ticks")

In [3]:
raw_fp='/home/ngrav/project/wearables/data/raw/MOD1000WomanActivityData20210707T213505Z-001/MOD 1000 Woman Activity Data/'
raw_md_fp='/home/ngrav/project/wearables/data/raw/MOD_Data_2021.csv'
raw_md = pd.read_csv(raw_md_fp, low_memory=False)

In [15]:
import importlib
importlib.reload(weardata)

<module 'wearables.scripts.data_v42' from '/home/ngrav/project/wearables/scripts/data_v42.py'>

In [16]:
rawdata = weardata.raw2df()

Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find light measurement.
Could not find

In [ ]:
len(np.unique([i.split('_')[0] for i in rawdata.exclude[k]] if isinstance(rawdata.exclude[k], list) else rawdata.exclude[k].split('_')[0]))))

In [43]:
print('Exclusion criteria (n_measurements, n_pts)')
for k in rawdata.exclude.keys():
    print('  {}: ({}, {})'.format(k, 
                               len(rawdata.exclude[k]), 
                               len(np.unique([i.split('_')[0] for i in rawdata.exclude[k]] if isinstance(rawdata.exclude[k], list) else rawdata.exclude[k].split('_')[0]))))


Exclusion criteria (n_measurements, n_pts)
  no_lux: (116, 110)
  chk_t: (7, 1)
  lt_1d: (7, 1)
  lt_max_t: (0, 0)
  corrupt_mtn: (7, 1)


In [29]:
rawdata.exclude['chk_t']

'1605_11'

In [28]:
rawdata.exclude['lt_1d']

'1325_32'

In [27]:
rawdata.exclude['corrupt_mtn']

'1168_24'

In [46]:
rawdata.exclude['lt_max_t'].append('123')

In [47]:
rawdata.exclude['lt_max_t']

['123']

# dev

In [20]:
# select a random file
ID = np.random.choice(list(rawdata.IDs.keys()))
file = rawdata.IDs[ID]
print(ID)

raw = pyActigraphy.io.read_raw_mtn(file)
print(raw.name)

2633_22
2633


# grab metadata and labels


In [31]:
raw_fp='/home/ngrav/project/wearables/data/raw/MOD1000WomanActivityData20210707T213505Z-001/MOD 1000 Woman Activity Data/'
raw_md_fp='/home/ngrav/project/wearables/data/raw/MOD_Data_2021.csv'
raw_md = pd.read_csv(raw_md_fp, low_memory=False)

In [48]:
# import variable from file
from wearables.data.processed.md_pp_specification import *

In [34]:
# chk that all PIDs have info in the metadata (produce themd_pp_specificationmpty list)
IDs = list(rawdata.IDs.keys())
[i for i in IDs if int(i.split('_')[0]) not in raw_md['record_id'].tolist()]

[]

In [63]:
raw_md.loc[raw_md['record_id'].isin([int(i.split('_')[0]) for i in IDs] ), :].shape

(1260, 3081)

In [69]:
with open('/home/ngrav/scratch/IDs.pkl', 'wb') as f:
    pickle.dump(list(rawdata.IDs.keys()), f, protocol=pickle.HIGHEST_PROTOCOL)
    f.close()

In [57]:
def delete_problematic_measurements(exclude_list=['no_lux', 'corrupt_mtn', 'lt_1d', 'chk_t']):
    for problem in exclude_list:
        if isinstance(rawdata.exclude[problem], list):
            for k in list(rawdata.exclude[problem]):
                print(k)
                del rawdata.IDs[k]
                del rawdata.data[k]
        else:
            print(rawdata.exclude[problem])
            del rawdata.data[rawdata.exclude[problem]]
            del rawdata.IDs[rawdata.exclude[problem]]

In [59]:
# filter out the IDs
delete_problematic_measurements()

2003_29


KeyError: '2003_29'

# train test splits


# visuzlize from train based on md